<a href="https://colab.research.google.com/github/Elizabeth-778/Angelman-syndrome/blob/main/Baseline_model%2C_combined_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install gprofiler-official

In [ ]:
import pandas as pd
import numpy as np
from functools import reduce
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from gprofiler import GProfiler
from collections import Counter
from sklearn.model_selection import RepeatedKFold, cross_val_score
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import accuracy_score, confusion_matrix

In [ ]:


gp = GProfiler(return_dataframe=True)

In [ ]:
#human
GSE67122 = pd.read_csv("GSE67122_counts_table_Control_AS_new_DeSeq2_All.txt", sep="\t", header=0)

In [ ]:
GSE67122['ALIAS'] = GSE67122['ALIAS'].str.upper()
GSE67122.head()

,ALIAS,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,Control_Neu_076_merged,Control_Neu_037A,Control_Neu_023,AS_Neu_078,AS_Neu_059_Lane2,AS_Neu_055_merged,SYMBOL,GENENAME
0,A1BG,39.526470,0.041215,0.493403,0.083532,0.933429,0.999849,46.865182,40.529608,28.710821,31.563869,45.251077,44.238266,A1BG,alpha-1-B glycoprotein
1,A1BG-AS1,2.464391,0.467102,2.008447,0.232569,0.816096,0.999849,3.712094,2.251645,0.000000,0.000000,7.725794,1.096817,A1BG-AS1,A1BG antisense RNA 1
2,A1CF,2.573820,-0.557334,1.865214,-0.298804,0.765089,0.999849,0.464012,9.006580,0.000000,1.578193,1.103685,3.290450,A1CF,APOBEC1 complementation factor
3,A2M,15.630055,1.451297,0.938394,1.546575,0.121966,0.999849,3.712094,13.509869,8.613246,45.767610,9.381321,12.796193,A2M,alpha-2-macroglobulin
4,A2M-AS1,8.877927,-1.312203,1.219467,-1.076046,0.281907,0.999849,20.880527,0.000000,17.226492,1.578193,8.829478,4.752872,A2M-AS1,A2M antisense RNA 1


In [ ]:
GSE67122_list=GSE67122['ALIAS'].tolist()

In [ ]:
gp = GProfiler(return_dataframe=True)
converted = gp.orth(organism='hsapiens',
            query=GSE67122_list,
            target='mmusculus')

In [ ]:
GSE67122['ALIAS'] = converted['ortholog_ensg']
GSE67122=GSE67122.rename(columns={'ALIAS': 'ENSEMBL'})

In [ ]:
#human
GSE120225 = pd.read_csv("GSE120225_RNAseq_expression_DeSeq2_All.txt", sep="\t", header=0)
GSE120225['ENSEMBL'] = GSE120225['ENSEMBL'].str.upper()

In [ ]:
GSE120225_list=GSE120225['ENSEMBL'].tolist()

In [ ]:
gp = GProfiler(return_dataframe=True)
converted = gp.orth(organism='hsapiens',
            query=GSE120225_list,
            target='mmusculus')

In [ ]:
GSE120225['ENSEMBL'] = converted['ortholog_ensg']

In [ ]:
GSE120225

,ENSEMBL,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,E1_Ha_1_S26_L004,E2_C30_1_S27_L004,E3_Ha_2_S28_L004,E4_C30_2_S29_L004,E5_H1_3_S30_L004,E6_C1_3_S31_L004,SYMBOL,GENENAME
0,N/A,44.137884,4.696338,0.731467,6.420434,1.358867e-10,0.000002,2.928697,4.268770,2.589324,32.147450,92.963933,129.929129,FGF7P6,fibroblast growth factor 7 pseudogene 6
1,ENSMUSG00000022860,426.023137,-1.308393,0.208735,-6.268213,3.652148e-10,0.000003,706.792113,519.722745,594.249828,259.935096,227.761635,247.677402,CHODL,chondrolectin
2,ENSMUSG00000052415,94.822351,4.254343,0.715080,5.949467,2.690171e-09,0.000016,6.833625,14.940695,6.473310,29.391954,164.546161,346.748363,TCHH,trichohyalin
3,ENSMUSG00000034573,706.360259,2.281332,0.410967,5.551135,2.838210e-08,0.000101,174.745564,180.355532,368.978651,517.114696,1583.175775,1413.791333,PTPN13,protein tyrosine phosphatase non-receptor type 13
4,ENSMUSG00000001076,605.631625,-1.832392,0.330087,-5.551242,2.836464e-08,0.000101,1373.558707,710.750202,752.198584,339.844471,284.469634,172.968153,C1QL4,complement C1q like 4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17841,ENSMUSG00000029076,7628.654928,-0.000050,0.151765,-0.000330,9.997368e-01,0.999916,7780.570642,7856.671147,7248.812168,6916.294242,8093.439989,7876.141379,BAG6,BAG cochaperone 6
17842,ENSMUSG00000004880,2607.714107,0.000041,0.166937,0.000246,9.998038e-01,0.999916,2481.582255,2973.198291,2367.936677,2500.153111,2622.512544,2700.901766,DGKZ,diacylglycerol kinase zeta
17843,N/A,3356.682864,0.000043,0.166409,0.000256,9.997958e-01,0.999916,3534.936800,3227.190104,3307.861241,2935.521434,3408.057776,3726.529826,C16orf72,chromosome 16 open reading frame 72
17844,ENSMUSG00000032497,672.819837,-0.000011,0.223230,-0.000049,9.999612e-01,0.999961,659.932968,769.445789,589.071180,653.052484,548.487203,816.929398,DPY19L2P2,DPY19L2 pseudogene 2


In [ ]:
#human
GSE146640 = pd.read_csv("GSE146640_allSamples_GSE146640_RawCounts_DeSeq2_All.txt", sep="\t", header=0)
GSE146640['ENSEMBL'] = GSE146640['ENSEMBL'].str.upper()

In [ ]:
GSE146640_list=GSE146640['ENSEMBL'].tolist()

In [ ]:
gp = GProfiler(return_dataframe=True)
converted = gp.orth(organism='hsapiens',
            query=GSE146640_list,
            target='mmusculus')

In [ ]:
GSE146640['ENSEMBL'] = converted['ortholog_ensg']

In [ ]:
GSE146640

,ENSEMBL,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,AS1_AGI,AS2_AS_D3,AS3_AS_ID,Ctr1_MCH2,Ctrl2_Ctrl,Ctrl3_Zip13,SYMBOL,GENENAME
0,N/A,7.421822,-19.561141,3.083664,-6.343473,2.246422e-10,0.000009,0.000000,0.000000,44.530932,0.000000,0.000000,0.000000,NaN,NaN
1,ENSMUSG00000026380,45.107097,-3.085915,0.507286,-6.083190,1.178145e-09,0.000011,87.104956,56.680666,97.774438,11.654253,8.903787,8.524485,TFCP2L1,transcription factor CP2 like 1
2,ENSMUSG00000021678,49.096102,-3.555667,0.586224,-6.065368,1.316517e-09,0.000011,110.162150,52.798428,109.391203,4.994680,7.284917,9.945233,F2RL1,F2R like trypsin receptor 1
3,N/A,6.453758,-19.380328,3.126775,-6.198185,5.711804e-10,0.000011,0.000000,0.000000,38.722550,0.000000,0.000000,0.000000,NaN,NaN
4,N/A,5.969726,-19.280454,3.153507,-6.113972,9.718118e-10,0.000011,0.000000,0.000000,35.818358,0.000000,0.000000,0.000000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41664,N/A,106.861881,-0.204270,0.437299,-0.467118,6.404153e-01,0.999996,144.747942,86.962117,113.263458,61.601051,111.702058,122.894661,LOC100132249,uncharacterized LOC100132249
41665,N/A,47.304219,0.252597,0.551567,0.457963,6.469792e-01,0.999996,43.552478,61.339351,25.169657,31.632972,51.803853,70.327002,PAX6-AS1,PAX6 antisense RNA 1
41666,N/A,34.986992,-0.781803,0.695032,-1.124844,2.606552e-01,0.999996,76.857314,24.069872,32.914167,11.654253,38.852890,25.573455,SCHIP1,schwannomin interacting protein 1
41667,ENSMUSG00000029586,33.605166,0.584071,1.039618,0.561813,5.742436e-01,0.999996,8.966687,6.988027,64.860271,28.303186,12.950963,79.561861,LSP1P5,LSP1 pseudogene 5


In [ ]:
#mouse
GSE16889 = pd.read_csv("GSE16889_mouse_hippocampus_DeSeq2_ExpressionTable_counts_DeSeq2_All.txt", sep="\t", header=0)

In [ ]:
GSE16889['ALIAS'] = GSE16889['ALIAS'].str.upper()

In [ ]:
GSE16889_list=GSE16889['ALIAS'].tolist()

In [ ]:
gp = GProfiler(return_dataframe=True)
converted_GSE16889 = gp.convert(organism='mmusculus',
            query=GSE16889_list,
            target_namespace='ENSG')

In [ ]:
GSE16889['ALIAS'] = converted_GSE16889['converted']

In [ ]:
GSE16889=GSE16889.rename(columns={'ALIAS': 'ENSEMBL'})
GSE16889=GSE16889.dropna()
GSE16889=GSE16889.reset_index(drop=True)

In [ ]:
#mouse
GSE130883 = pd.read_csv("GSE130883_ExpressionTable_DeSeq2_All.txt", sep="\t", header=0)

In [ ]:
GSE130883['ENSEMBL'] = GSE130883['ENSEMBL'].str.upper()

In [ ]:
#mouse
GSE137217_p2 = pd.read_csv("GSE137217_Cortex_counts_p2_maternalParentalOrigin_new_DeSeq2_All.txt", sep="\t", header=0)

In [ ]:
GSE137217_p2['ACCNUM'] = GSE137217_p2['ACCNUM'].str.upper()

In [ ]:
accnum137217_p2_list=GSE137217_p2['ACCNUM'].tolist()

In [ ]:
gp = GProfiler(return_dataframe=True)
converted_mouse_137217_p2 = gp.convert(organism='mmusculus',
            query=accnum137217_p2_list,
            target_namespace='ENSG')

In [ ]:
GSE137217_p2['ACCNUM'] = converted_mouse_137217_p2['converted']

In [ ]:
GSE137217_p2=GSE137217_p2.rename(columns={'ACCNUM': 'ENSEMBL'})

In [ ]:
GSE137217_p2

,ENSEMBL,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,JL602,JL601,JL600,JL605,JL604,JL603,SYMBOL,GENENAME
0,ENSMUSG00000063434,22.777929,-0.004948,0.414629,-0.011933,0.990479,0.999928,24.326661,25.682747,18.289775,27.430061,22.180483,18.757846,Sorcs3,sortilin-related VPS10 domain containing recep...
1,ENSMUSG00000030852,8.064810,-0.440830,0.695017,-0.634272,0.525903,0.999928,9.009874,5.707277,13.209282,5.486012,8.065630,6.910786,Tacc2,"transforming, acidic coiled-coil containing pr..."
2,ENSMUSG00000096054,94.465695,-0.340287,0.251428,-1.353417,0.175922,0.999928,137.851079,97.974923,80.271790,98.748220,90.738340,61.209815,Syne1,"spectrin repeat containing, nuclear envelope 1"
3,ENSMUSG00000030518,520.935367,0.230023,0.137713,1.670307,0.094859,0.999928,401.840401,570.727706,466.389262,536.531998,521.241354,628.881482,Fam189a1,"family with sequence similarity 189, member A1"
4,ENSMUSG00000030536,221.890075,0.055961,0.189476,0.295346,0.767730,0.999928,207.227113,277.754150,167.656271,204.079656,273.223224,201.400035,Iqgap1,IQ motif containing GTPase activating protein 1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8761,None,6.108312,0.686934,0.815697,0.842144,0.399707,0.999928,8.108887,3.804851,2.032197,8.777620,5.041019,8.885296,NaN,NaN
8762,ENSMUSG00000047139,2781.267974,0.084655,0.097963,0.864151,0.387505,0.999928,2604.754708,2545.445567,2948.718168,3125.929777,2911.692515,2551.067109,Cd24a,CD24a antigen
8763,ENSMUSG00000038446,147.207978,-0.029780,0.176525,-0.168698,0.866034,0.999928,136.049104,157.901332,152.414792,144.830723,134.091103,157.960812,Cdc40,cell division cycle 40
8764,ENSMUSG00000063804,7.239950,-0.299956,0.722398,-0.415223,0.677979,0.999928,7.207900,6.658490,10.160986,5.486012,5.041019,8.885296,Lin28b,lin-28 homolog B (C. elegans)


In [ ]:
#mouse
GSE137217_p9 = pd.read_csv("GSE137217_Cortex_counts_p9_maternalParentalOrigin_new_DeSeq2_All.txt", sep="\t", header=0)

In [ ]:
GSE137217_p9['ACCNUM'] = GSE137217_p9['ACCNUM'].str.upper()

In [ ]:
accnum137217_p9_list=GSE137217_p9['ACCNUM'].tolist()

In [ ]:
gp = GProfiler(return_dataframe=True)
converted_mouse_137217_p9 = gp.convert(organism='mmusculus',
            query=accnum137217_p9_list,
            target_namespace='ENSG')

In [ ]:
GSE137217_p9['ACCNUM'] = converted_mouse_137217_p9['converted']

In [ ]:
GSE137217_p9=GSE137217_p9.rename(columns={'ACCNUM': 'ENSEMBL'})

In [ ]:
hypothalamus_p2 = pd.read_csv("Hypothalamus_p2_ParentOriginMaternal_ForDeSeq2_Result.txt", sep="\t", header=0)

In [ ]:
hypothalamus_p2['Id'] = hypothalamus_p2['Id'].str.upper()

In [ ]:
hypothalamus_p2_list=hypothalamus_p2['Id'].tolist()

In [ ]:
gp = GProfiler(return_dataframe=True)
converted_hypothalamus_p2 = gp.convert(organism='mmusculus',
            query=hypothalamus_p2_list,
            target_namespace='ENSG')

In [ ]:
hypothalamus_p2['Id'] = converted_hypothalamus_p2['converted']

In [ ]:
hypothalamus_p2=hypothalamus_p2.rename(columns={'Id': 'ENSEMBL'})

In [ ]:
#mouse
hypothalamus_p9 = pd.read_csv("GSE137217_hypothalamus_p9_maternal_DeSeq2_All.txt", sep="\t", header=0)

In [ ]:
hypothalamus_p9['ACCNUM'] = hypothalamus_p9['ACCNUM'].str.upper()

In [ ]:
hypothalamus_p9_list=hypothalamus_p9['ACCNUM'].tolist()

In [ ]:
gp = GProfiler(return_dataframe=True)
converted_hypothalamus_p9 = gp.convert(organism='mmusculus',
            query=hypothalamus_p9_list,
            target_namespace='ENSG')

In [ ]:
hypothalamus_p9['ACCNUM'] = converted_hypothalamus_p9['converted']

In [ ]:
hypothalamus_p9=hypothalamus_p9.rename(columns={'ACCNUM': 'ENSEMBL'})

In [ ]:
GSE137217_p2_genes = set(GSE137217_p2['ENSEMBL'])
GSE137217_p9_genes = set(GSE137217_p9['ENSEMBL'])
hypothalamus_p2_genes = set(hypothalamus_p2['ENSEMBL'])
hypothalamus_p9_genes = set(hypothalamus_p9['ENSEMBL'])
GSE16889_genes = set(GSE16889['ENSEMBL'])
GSE130883_genes = set(GSE130883['ENSEMBL'])
GSE67122_genes = set(GSE67122['ENSEMBL'])
GSE120225_genes = set(GSE120225['ENSEMBL'])
GSE146640_genes = set(GSE146640['ENSEMBL'])

In [ ]:
human_mouse_genes_intersection = GSE137217_p2_genes.intersection(GSE137217_p9_genes,
                                                                     hypothalamus_p2_genes,
                                                                     hypothalamus_p9_genes,
                                                                     GSE16889_genes,
                                                                     GSE130883_genes,
                                                                    GSE67122_genes,
                                                                    GSE120225_genes,
                                                                    GSE146640_genes
                                                           )

In [ ]:
len(human_mouse_genes_intersection)

2399

In [ ]:
GSE137217_p2_no_duplicates = GSE137217_p2.drop_duplicates(subset=['ENSEMBL'])
filtered_GSE137217_p2 = GSE137217_p2_no_duplicates[GSE137217_p2_no_duplicates['ENSEMBL'].isin(human_mouse_genes_intersection)]

In [ ]:
filtered_GSE137217_p2 = filtered_GSE137217_p2.drop(columns=['baseMean',
                                                                              'log2FoldChange',
                                                                              'lfcSE',
                                                                              'stat',
                                                                              'pvalue',
                                                                              'padj',
                                                                              'SYMBOL',
                                                                              'GENENAME'

                                                                              ])

In [ ]:
filtered_GSE137217_p2 = filtered_GSE137217_p2.set_index('ENSEMBL').T.reset_index()
filtered_GSE137217_p2['Disease status'] = [0,0,0,1,1,1]
filtered_GSE137217_p2 = filtered_GSE137217_p2.drop(columns=['index'])

In [ ]:
GSE137217_p9_no_duplicates=GSE137217_p9.drop_duplicates(subset=['ENSEMBL'])
filtered_GSE137217_p9 = GSE137217_p9_no_duplicates[GSE137217_p9_no_duplicates['ENSEMBL'].isin(human_mouse_genes_intersection)]

In [ ]:
filtered_GSE137217_p9 = filtered_GSE137217_p9.drop(columns=['baseMean',
                                                                              'log2FoldChange',
                                                                              'lfcSE',
                                                                              'stat',
                                                                              'pvalue',
                                                                              'padj',
                                                                              'SYMBOL',
                                                                              'GENENAME'

                                                                              ])

In [ ]:
filtered_GSE137217_p9 = filtered_GSE137217_p9.set_index('ENSEMBL').T.reset_index()
filtered_GSE137217_p9['Disease status'] = [0,0,0,1,1,1]
filtered_GSE137217_p9 = filtered_GSE137217_p9.drop(columns=['index'])

In [ ]:
hypothalamus_p2_no_duplicates=hypothalamus_p2.drop_duplicates(subset=['ENSEMBL'])
filtered_hypothalamus_p2 = hypothalamus_p2_no_duplicates[hypothalamus_p2_no_duplicates['ENSEMBL'].isin(human_mouse_genes_intersection)]

In [ ]:
filtered_hypothalamus_p2 = filtered_hypothalamus_p2.set_index('ENSEMBL').T.reset_index()
filtered_hypothalamus_p2['Disease status'] = [0,0,0,1,1,1]
filtered_hypothalamus_p2 = filtered_hypothalamus_p2.drop(columns=['index'])

In [ ]:
hypothalamus_p9_no_duplicates=hypothalamus_p9.drop_duplicates(subset=['ENSEMBL'])
filtered_hypothalamus_p9 = hypothalamus_p9_no_duplicates[hypothalamus_p9_no_duplicates['ENSEMBL'].isin(human_mouse_genes_intersection)]

In [ ]:
filtered_hypothalamus_p9 = filtered_hypothalamus_p9.drop(columns=['baseMean',
                                                                              'log2FoldChange',
                                                                              'lfcSE',
                                                                              'stat',
                                                                              'pvalue',
                                                                              'padj',
                                                                              'SYMBOL',
                                                                              'GENENAME'

                                                                              ])

In [ ]:
filtered_hypothalamus_p9 = filtered_hypothalamus_p9.set_index('ENSEMBL').T.reset_index()
filtered_hypothalamus_p9['Disease status'] = [0,0,0,1,1,1]
filtered_hypothalamus_p9 = filtered_hypothalamus_p9.drop(columns=['index'])

In [ ]:
GSE16889_no_duplicates=GSE16889.drop_duplicates(subset=['ENSEMBL'])
filtered_GSE16889 = GSE16889_no_duplicates[GSE16889_no_duplicates['ENSEMBL'].isin(human_mouse_genes_intersection)]

In [ ]:
filtered_GSE16889 = filtered_GSE16889.drop(columns=['baseMean',
                                                                              'log2FoldChange',
                                                                              'lfcSE',
                                                                              'stat',
                                                                              'pvalue',
                                                                              'padj',
                                                                              'SYMBOL',
                                                                              'GENENAME'

                                                                              ])

In [ ]:
filtered_GSE16889 = filtered_GSE16889.set_index('ENSEMBL').T.reset_index()
filtered_GSE16889['Disease status'] = [0,0,0,1,1,1]
filtered_GSE16889 = filtered_GSE16889.drop(columns=['index'])

In [ ]:
GSE130883_no_duplicates=GSE130883.drop_duplicates(subset=['ENSEMBL'])
filtered_GSE130883 = GSE130883_no_duplicates[GSE130883_no_duplicates['ENSEMBL'].isin(human_mouse_genes_intersection)]

In [ ]:
filtered_GSE130883 = filtered_GSE130883.drop(columns=['baseMean',
                                                                              'log2FoldChange',
                                                                              'lfcSE',
                                                                              'stat',
                                                                              'pvalue',
                                                                              'padj',
                                                                              'SYMBOL',
                                                                              'GENENAME'

                                                                              ])

In [ ]:
filtered_GSE130883 = filtered_GSE130883.set_index('ENSEMBL').T.reset_index()
filtered_GSE130883['Disease status'] = [0,0,0,0,0,0,1,1,1,1,1,1]
filtered_GSE130883 = filtered_GSE130883.drop(columns=['index'])

In [ ]:
GSE67122_no_duplicates=GSE67122.drop_duplicates(subset=['ENSEMBL'])
filtered_GSE67122 = GSE67122_no_duplicates[GSE67122_no_duplicates['ENSEMBL'].isin(human_mouse_genes_intersection)]

In [ ]:
filtered_GSE67122 = filtered_GSE67122.drop(columns=['baseMean',
                                                                              'log2FoldChange',
                                                                              'lfcSE',
                                                                              'stat',
                                                                              'pvalue',
                                                                              'padj',
                                                                              'SYMBOL',
                                                                              'GENENAME'

                                                                              ])

In [ ]:
filtered_GSE67122 = filtered_GSE67122.set_index('ENSEMBL').T.reset_index()
filtered_GSE67122['Disease status'] = [0,0,0,1,1,1]
filtered_GSE67122 = filtered_GSE67122.drop(columns=['index'])

In [ ]:
GSE120225_no_duplicates=GSE120225.drop_duplicates(subset=['ENSEMBL'])
filtered_GSE120225 = GSE120225_no_duplicates[GSE120225_no_duplicates['ENSEMBL'].isin(human_mouse_genes_intersection)]

In [ ]:
filtered_GSE120225 = filtered_GSE120225.drop(columns=['baseMean',
                                                                              'log2FoldChange',
                                                                              'lfcSE',
                                                                              'stat',
                                                                              'pvalue',
                                                                              'padj',
                                                                              'SYMBOL',
                                                                              'GENENAME'

                                                                              ])

In [ ]:
filtered_GSE120225 = filtered_GSE120225.set_index('ENSEMBL').T.reset_index()
filtered_GSE120225['Disease status'] = [0,0,0,1,1,1]
filtered_GSE120225 = filtered_GSE120225.drop(columns=['index'])

In [ ]:
GSE146640_no_duplicates=GSE146640.drop_duplicates(subset=['ENSEMBL'])
filtered_GSE146640 = GSE146640_no_duplicates[GSE146640_no_duplicates['ENSEMBL'].isin(human_mouse_genes_intersection)]

In [ ]:
filtered_GSE146640 = filtered_GSE146640.drop(columns=['baseMean',
                                                                              'log2FoldChange',
                                                                              'lfcSE',
                                                                              'stat',
                                                                              'pvalue',
                                                                              'padj',
                                                                              'SYMBOL',
                                                                              'GENENAME'

                                                                              ])

In [ ]:
filtered_GSE146640 = filtered_GSE146640.set_index('ENSEMBL').T.reset_index()
filtered_GSE146640['Disease status'] = [0,0,0,1,1,1]
filtered_GSE146640 = filtered_GSE146640.drop(columns=['index'])

In [ ]:
concatenated_human_mouse_df = pd.concat([filtered_GSE16889,
                                   filtered_hypothalamus_p9,
                                   filtered_hypothalamus_p2,
                                   filtered_GSE137217_p9,
                                   filtered_GSE137217_p2,
                                   filtered_GSE130883,
                                   filtered_GSE67122,
                                   filtered_GSE120225,
                                   filtered_GSE146640

                                   ])

In [ ]:
concatenated_human_mouse_df

ENSEMBL,ENSMUSG00000058706,ENSMUSG00000046999,ENSMUSG00000032551,ENSMUSG00000087651,ENSMUSG00000043629,ENSMUSG00000032666,ENSMUSG00000052794,ENSMUSG00000038323,ENSMUSG00000040822,ENSMUSG00000054091,...,ENSMUSG00000036918,ENSMUSG00000042734,ENSMUSG00000071660,ENSMUSG00000026803,ENSMUSG00000027394,ENSMUSG00000022442,ENSMUSG00000016757,ENSMUSG00000031028,ENSMUSG00000001525,Disease status
0,274.102105,82.123977,97.055609,75.724706,73.591616,962.023729,103.454880,241.039205,353.026446,189.845038,...,909.763017,1130.537864,5307.128689,5307.128689,1161.467673,714.585254,714.585254,535.405668,876.700117,0
1,253.558361,84.173062,126.779181,70.663806,113.269924,1056.839562,80.016368,340.848944,553.879535,158.993562,...,655.718548,1005.920055,3750.377562,3750.377562,491.529118,816.790458,816.790458,650.522680,941.491291,0
2,255.668724,103.980103,160.251688,97.863626,113.766466,989.645921,116.213056,310.717014,522.347106,146.795439,...,604.307893,1066.713527,3627.070651,3627.070651,382.891438,748.656741,748.656741,618.987437,956.616947,0
3,232.194995,154.455701,77.739293,74.670637,68.533324,1043.343148,74.670637,235.263651,353.918362,135.020878,...,813.193924,1134.379952,5520.512713,5520.512713,735.454630,783.530246,783.530246,527.808886,936.963062,1
4,260.916294,102.132372,98.940735,99.738645,68.620187,994.194809,95.749099,247.351838,305.599207,167.560923,...,1006.163446,1111.487455,5121.778875,5121.778875,1023.717448,746.842970,746.842970,459.595674,873.710526,1
5,270.357859,152.351497,98.632183,75.735426,70.451559,846.299357,118.006362,233.370790,339.048130,153.232142,...,1108.731416,1097.283037,5497.863565,5497.863565,1136.031395,757.354263,757.354263,501.967361,902.660605,1
0,15.545483,5.360511,5.360511,17.153636,17.689687,109.354429,29.482812,23.586249,8.040767,108.818378,...,6.968665,36.451476,55.213265,15.545483,183.865535,3.216307,21.442045,99.705509,6.968665,0
1,9.451811,3.544429,4.725906,10.633288,14.177717,113.421735,37.807245,25.992481,9.451811,88.610730,...,20.085099,40.170198,61.436773,12.996240,199.669513,2.362953,29.536910,126.417975,8.270335,0
2,12.888070,3.222017,5.370029,12.888070,23.628128,93.438507,28.998157,27.924152,11.814064,67.662367,...,6.444035,34.368187,35.442192,13.962076,170.766927,6.444035,19.332105,109.548595,3.222017,0
3,22.829108,2.853639,5.707277,19.975470,8.560916,104.157806,18.548650,25.682747,7.134096,77.048240,...,7.134096,58.499590,39.950939,11.414554,182.632866,8.560916,24.255927,114.145541,7.134096,1


In [ ]:
scaler = MinMaxScaler()

normalized_human_mouse_features = scaler.fit_transform(concatenated_human_mouse_df)
normalized_human_mouse_df = pd.DataFrame(normalized_human_mouse_features, columns=concatenated_human_mouse_df.columns)
normalized_human_mouse_df

ENSEMBL,ENSMUSG00000058706,ENSMUSG00000046999,ENSMUSG00000032551,ENSMUSG00000087651,ENSMUSG00000043629,ENSMUSG00000032666,ENSMUSG00000052794,ENSMUSG00000038323,ENSMUSG00000040822,ENSMUSG00000054091,...,ENSMUSG00000036918,ENSMUSG00000042734,ENSMUSG00000071660,ENSMUSG00000026803,ENSMUSG00000027394,ENSMUSG00000022442,ENSMUSG00000016757,ENSMUSG00000031028,ENSMUSG00000001525,Disease status
0,0.342681,0.030469,0.103700,0.034596,0.220314,0.800756,0.034469,0.269982,0.090154,0.024863,...,0.761604,0.707653,0.961326,0.830502,0.418120,0.874529,0.874563,0.314740,0.916459,0.0
1,0.316997,0.031238,0.135458,0.032284,0.339100,0.879924,0.026660,0.382712,0.141739,0.020735,...,0.548932,0.629650,0.679179,0.586797,0.176322,1.000000,1.000000,0.382412,0.984188,0.0
2,0.319635,0.038677,0.171222,0.044711,0.340586,0.823819,0.038720,0.348680,0.133640,0.019103,...,0.505893,0.667703,0.656831,0.567494,0.137112,0.916357,0.916379,0.363874,1.000000,0.0
3,0.290289,0.057635,0.083061,0.034115,0.205170,0.868655,0.024879,0.263459,0.090383,0.017528,...,0.680761,0.710058,1.000000,0.863906,0.264361,0.959169,0.959179,0.310274,0.979455,1.0
4,0.326196,0.037983,0.105714,0.045567,0.205430,0.827617,0.031902,0.277112,0.077973,0.021881,...,0.842305,0.695729,0.927733,0.801486,0.368402,0.914130,0.914153,0.270175,0.913334,1.0
5,0.338000,0.056845,0.105384,0.034601,0.210913,0.704129,0.039318,0.261321,0.086564,0.019964,...,0.928169,0.686838,0.995895,0.860361,0.408939,0.927034,0.927053,0.295083,0.943597,1.0
6,0.019435,0.001638,0.005727,0.007837,0.052958,0.088803,0.009823,0.024381,0.001551,0.014023,...,0.005834,0.022817,0.009463,0.002121,0.065279,0.001228,0.023861,0.058612,0.007285,0.0
7,0.011817,0.000956,0.005049,0.004858,0.042444,0.092199,0.012597,0.027098,0.001914,0.011319,...,0.016814,0.025144,0.010591,0.001721,0.070983,0.000180,0.033796,0.074315,0.008645,0.0
8,0.016113,0.000835,0.005738,0.005888,0.070736,0.075513,0.009662,0.029280,0.002521,0.008517,...,0.005395,0.021513,0.005880,0.001873,0.060551,0.005190,0.021272,0.064398,0.003368,0.0
9,0.028541,0.000696,0.006098,0.009126,0.025629,0.084464,0.006180,0.026748,0.001319,0.009772,...,0.005972,0.036617,0.006697,0.001474,0.064834,0.007789,0.027315,0.067101,0.007458,1.0


In [ ]:
outliers_indices_dict = {}
for gene in normalized_human_mouse_df.columns:

    Q1 = normalized_human_mouse_df[gene].quantile(0.25)
    Q3 = normalized_human_mouse_df[gene].quantile(0.75)


    IQR = Q3 - Q1


    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR


    outliers_mask = (normalized_human_mouse_df[gene] < lower_bound) | (normalized_human_mouse_df[gene] > upper_bound)


    outlier_indices = normalized_human_mouse_df[gene][outliers_mask].index.tolist()


    if outlier_indices:
        outliers_indices_dict[gene] = outlier_indices

In [ ]:


all_indices = [index for indices in outliers_indices_dict.values() for index in indices]


index_frequency = Counter(all_indices)


most_common_indices = index_frequency.most_common()

print("Most common indices and their frequencies:")
for index, count in most_common_indices:
    print(f"Index: {index}, Frequency: {count}")

Most common indices and their frequencies:
Index: 53, Frequency: 860
Index: 50, Frequency: 854
Index: 51, Frequency: 853
Index: 52, Frequency: 850
Index: 49, Frequency: 823
Index: 48, Frequency: 821
Index: 4, Frequency: 758
Index: 0, Frequency: 756
Index: 5, Frequency: 756
Index: 1, Frequency: 755
Index: 2, Frequency: 753
Index: 3, Frequency: 749
Index: 58, Frequency: 448
Index: 55, Frequency: 447
Index: 56, Frequency: 438
Index: 54, Frequency: 435
Index: 59, Frequency: 432
Index: 57, Frequency: 417
Index: 44, Frequency: 275
Index: 47, Frequency: 265
Index: 43, Frequency: 264
Index: 45, Frequency: 263
Index: 46, Frequency: 259
Index: 42, Frequency: 250
Index: 18, Frequency: 151
Index: 22, Frequency: 145
Index: 23, Frequency: 145
Index: 21, Frequency: 142
Index: 19, Frequency: 139
Index: 20, Frequency: 139
Index: 27, Frequency: 110
Index: 29, Frequency: 108
Index: 24, Frequency: 104
Index: 25, Frequency: 104
Index: 28, Frequency: 102
Index: 26, Frequency: 96
Index: 12, Frequency: 45
Ind

In [ ]:

indices_to_drop = [53,50,51,52,49,48,4,0,5,1,2,3]
normalized_human_mouse_df.drop(indices_to_drop, inplace=True)

normalized_human_mouse_df.reset_index(drop=True, inplace=True)

In [ ]:
y_human_mouse = normalized_human_mouse_df['Disease status']
X_human_mouse = normalized_human_mouse_df.drop(columns=['Disease status'], axis=1)
X_human_mouse_train, X_human_mouse_test, y_human_mouse_train, y_human_mouse_test = train_test_split(X_human_mouse, y_human_mouse, test_size=0.2, random_state=42)

In [ ]:
loo = LeaveOneOut()

In [ ]:

rkf = RepeatedKFold(n_splits=5, n_repeats=5,random_state=42)
model = LogisticRegression(max_iter=1000)
y_human_mouse = normalized_human_mouse_df['Disease status']
X_human_mouse = normalized_human_mouse_df.drop(columns=['Disease status'], axis=1)

cv_scores = cross_val_score(model, X_human_mouse, y_human_mouse, cv=rkf)

print("Mean of Cross-Validation Scores of Logistic regression:", np.mean(cv_scores))

Mean of Cross-Validation Scores of Logistic regression: 0.24222222222222223


In [ ]:
accuracy_scores = []
y = normalized_human_mouse_df['Disease status']
X = normalized_human_mouse_df.drop(columns=['Disease status'], axis=1).values
model = LogisticRegression(max_iter=1000)
for train_index, test_index in loo.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]


    model.fit(X_train, y_train)


    y_pred = model.predict(X_test)


    accuracy = accuracy_score(y_test, y_pred)
    accuracy_scores.append(accuracy)


average_accuracy = np.mean(accuracy_scores)
print("Average Jackknife Accuracy for Logistic regression:", average_accuracy)

Average Jackknife Accuracy for Logistic regression: 0.14583333333333334


In [ ]:

rkf = RepeatedKFold(n_splits=5, n_repeats=5,random_state=42)
model = SVC(kernel='sigmoid')
y_human_mouse = normalized_human_mouse_df['Disease status']
X_human_mouse = normalized_human_mouse_df.drop(columns=['Disease status'], axis=1)

cv_scores = cross_val_score(model, X_human_mouse, y_human_mouse, cv=rkf)


print("Mean of Cross-Validation Scores of SVM:", np.mean(cv_scores))

Mean of Cross-Validation Scores of SVM: 0.30844444444444447


In [ ]:
accuracy_scores = []
y = normalized_human_mouse_df['Disease status']
X = normalized_human_mouse_df.drop(columns=['Disease status'], axis=1).values
model = SVC(kernel='sigmoid')
for train_index, test_index in loo.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]


    model.fit(X_train, y_train)


    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    accuracy_scores.append(accuracy)

average_accuracy = np.mean(accuracy_scores)
print("Average Jackknife Accuracy for SVM:", average_accuracy)

Average Jackknife Accuracy for SVM: 0.0


In [ ]:

rkf = RepeatedKFold(n_splits=5, n_repeats=5,random_state=42)
model = RandomForestClassifier()
y_human_mouse = normalized_human_mouse_df['Disease status']
X_human_mouse = normalized_human_mouse_df.drop(columns=['Disease status'], axis=1)

cv_scores = cross_val_score(model, X_human_mouse, y_human_mouse, cv=rkf)


print("Mean of Cross-Validation Scores of Random forest:", np.mean(cv_scores))

Mean of Cross-Validation Scores of Random forest: 0.2413333333333333


In [ ]:
accuracy_scores = []
y = normalized_human_mouse_df['Disease status']
X = normalized_human_mouse_df.drop(columns=['Disease status'], axis=1).values
model = RandomForestClassifier()
for train_index, test_index in loo.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]


    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)


    accuracy = accuracy_score(y_test, y_pred)
    accuracy_scores.append(accuracy)


average_accuracy = np.mean(accuracy_scores)
print("Average Jackknife Accuracy for Random forest:", average_accuracy)

Average Jackknife Accuracy for Random forest: 0.20833333333333334
